<a href="https://colab.research.google.com/github/SidKataria/501A4/blob/master/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# @author Siddharth Kataria
# UCID: 30000880
# Fall 2019
# CPSC 501
# Assignment 4 - Part 3

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow_core.python.keras import regularizers

In [53]:
from google.colab import files
uploaded = files.upload()

Saving heart.csv to heart (1).csv


In [54]:
import pandas as pd
import numpy as np

df = pd.read_csv('heart.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]
print(test)
print(train)

     row.names  sbp  tobacco   ldl  ...  alcohol age  chd     split
0            1  160    12.00  5.73  ...    97.20  52    1 -0.098444
3            4  170     7.50  6.41  ...    24.26  58    1 -1.775224
7            8  114     4.08  4.59  ...     6.72  58    1 -1.982076
11          12  134    14.10  4.44  ...     0.00  40    1  1.534408
12          13  118     0.00  1.88  ...     0.00  17    0 -0.720040
..         ...  ...      ...   ...  ...      ...  ..  ...       ...
442        444  166     6.00  8.80  ...    43.20  52    0  0.114048
454        456  146     0.64  4.82  ...     8.23  39    1  0.012528
456        458  170     0.40  4.11  ...     2.06  57    0 -0.351867
457        459  214     0.40  5.98  ...     0.00  58    0 -0.250826
461        463  132     0.00  4.82  ...     0.00  46    1 -0.287198

[126 rows x 12 columns]
     row.names  sbp  tobacco    ldl  ...  alcohol age  chd     split
1            2  144     0.01   4.41  ...     2.06  63    1 -0.742985
2            3  118  

In [0]:
from google.colab import files
test.to_csv('heart_test.csv')
train.to_csv('heart_train.csv')
files.download('heart_test.csv')
files.download('heart_train.csv')

In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [0]:
np.set_printoptions(precision=3, suppress=True)

Checking the csv


In [58]:
!head {train_file_path}
!head {test_file_path}

,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd,split
1,2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1,-0.7429847363176832
2,3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0,-0.27445289166690107
4,5,134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1,0.302100559068745
5,6,132,6.2,6.47,36.21,Present,62,30.77,14.14,45,0,0.789408477831719
6,7,142,4.05,3.38,16.2,Absent,59,20.81,2.62,38,0,-0.3345650311190216
8,9,114,0.0,3.83,19.4,Present,49,24.86,2.49,29,0,-1.1871482180668584
9,10,132,0.0,5.8,30.96,Present,69,30.11,0.0,53,1,2.1592838137006525
10,11,206,6.0,2.95,32.27,Absent,72,26.81,56.06,60,1,0.9751351390166565
13,14,132,0.0,1.87,17.21,Absent,49,23.63,0.97,15,0,1.4708476054596495
,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd,split
0,1,160,12.0,5.73,23.11,Present,49,25.3,97.2,52,1,-0.09844384692212212
3,4,170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1,-1.7752242229959088
7,8,114,4.08,4.59,14.6,Present,62,23.11,6.72,58,1,-1.9820763614832215

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

Reading the csv data

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=10,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

SELECT_COLUMNS = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']

raw_train_data = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_file_path, select_columns=SELECT_COLUMNS)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Checking training and test data

In [62]:
print("Train: ")
show_batch(raw_train_data)
print("Test: ")
show_batch(raw_test_data)

Train: 
sbp                 : [126 164 150 142 118 164 116 174 178 108]
tobacco             : [ 0.   12.    0.    0.    1.05 13.02 31.2   2.02  0.95  0.8 ]
ldl                 : [5.98 3.91 4.99 4.19 3.16 6.26 3.17 6.57 4.75 2.47]
adiposity           : [29.06 19.59 27.73 18.04 12.98 29.38 14.99 31.9  21.06 17.53]
famhist             : [b'Present' b'Absent' b'Absent' b'Absent' b'Present' b'Present' b'Absent'
 b'Present' b'Absent' b'Absent']
typea               : [56 51 57 56 46 47 47 50 49 47]
obesity             : [25.39 23.44 30.92 23.65 22.09 22.75 19.4  28.75 23.74 22.18]
alcohol             : [11.52 19.75  8.33 20.78 16.35 37.03 49.06 11.83 24.69  0.  ]
age                 : [64 39 24 42 31 54 59 64 61 55]
Test: 
sbp                 : [122 142 118 103 124 174 101 142 142 148]
tobacco             : [0.   0.28 0.   0.03 6.   9.45 0.48 0.   7.44 0.  ]
ldl                 : [3.08 1.8  1.88 4.21 5.21 5.13 7.26 3.54 5.52 5.32]
adiposity           : [16.3  21.03 10.05 18.96 33.02 35.54 13.

In [0]:
train_batch, labels_batch = next(iter(raw_train_data)) 
test_batch, labels_batch = next(iter(raw_test_data)) 

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

Putting in training and test data


In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']
FEATURES = 8

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

Checking packed training and test data

In [67]:
print("Train: ")
show_batch(packed_train_data)
print("Test: ")
show_batch(packed_test_data)

Train: 
famhist             : [b'Absent' b'Present' b'Present' b'Absent' b'Absent' b'Present' b'Absent'
 b'Present' b'Absent' b'Present']
numeric             : [[128.     2.24   2.83  26.48  48.    23.96  47.42  27.  ]
 [176.     6.     3.98  17.2   52.    21.07   4.11  61.  ]
 [162.     7.4    8.55  24.65  64.    25.71   5.86  58.  ]
 [110.    12.16   4.99  28.56  44.    27.14  21.6   55.  ]
 [122.     0.     3.37  16.1   67.    21.06   0.    32.  ]
 [136.     3.96   2.76  30.28  50.    34.42  18.51  38.  ]
 [158.     1.02   6.33  23.88  66.    22.13  24.99  46.  ]
 [136.     3.99   2.58  16.38  53.    22.41  27.67  36.  ]
 [148.     4.5   10.49  33.27  50.    25.92   2.06  53.  ]
 [122.     0.     5.75  30.9   46.    29.01   4.11  42.  ]]
Test: 
famhist             : [b'Absent' b'Absent' b'Present' b'Absent' b'Absent' b'Present' b'Absent'
 b'Present' b'Absent' b'Absent']
numeric             : [[130.     0.56   3.3   30.86  49.    27.52  33.33  45.  ]
 [138.    12.     5.13  28.34  59

Creating Branches

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

Checking normalized data

In [69]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000
mean,137.803571,3.616012,4.681280,24.957679,53.285714,25.975506,17.561815,42.288690
std,20.403617,4.625679,2.103097,7.671883,9.927284,4.098951,24.320584,14.427896
min,102.000000,0.000000,0.980000,6.740000,20.000000,17.750000,0.000000,15.000000
25%,124.000000,0.077500,3.195000,19.460000,47.000000,22.940000,0.637500,30.750000
50%,134.000000,2.000000,4.235000,25.700000,54.000000,25.740000,7.780000,44.000000
75%,146.000000,5.400000,5.800000,30.530000,60.000000,28.347500,24.322500,55.000000
max,218.000000,31.200000,15.330000,42.170000,78.000000,46.580000,147.190000,64.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

Final normalized data

In [72]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(8,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fcd5ddea840>, mean=array([137.804,   3.616,   4.681,  24.958,  53.286,  25.976,  17.562,
        42.289]), std=array([20.404,  4.626,  2.103,  7.672,  9.927,  4.099, 24.321, 14.428])))

In [73]:
train_batch['numeric']
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

array([[-0.382,  0.792, -0.533, -0.217, -3.353,  0.201,  1.411,  1.089],
       [ 0.696,  1.402,  0.014, -0.04 ,  1.18 ,  0.057,  0.287,  1.02 ],
       [-0.873, -0.782, -1.075, -1.082, -0.734, -1.419, -0.722, -1.753],
       [-0.579,  0.04 , -0.381,  0.891,  0.374,  1.111, -0.722, -0.852],
       [-0.48 , -0.773,  1.683,  0.419,  1.18 ,  0.065, -0.24 , -1.268],
       [ 0.108, -0.652,  0.418,  1.099,  0.475,  0.296, -0.722,  0.881],
       [-0.088, -0.782,  0.152,  0.342, -0.432,  0.394, -0.662, -0.228],
       [ 2.754, -0.23 ,  1.05 ,  1.163,  1.583,  0.818, -0.722, -0.089],
       [-0.284,  0.792, -0.552, -1.646,  0.676, -1.585, -0.637,  0.95 ],
       [ 1.186, -0.782,  0.194, -0.047,  1.079,  0.179, -0.565, -1.683]],
      dtype=float32)

Categorizing data

In [74]:
CATEGORIES = {
    'famhist': ['Present', 'Absent']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='famhist', vocabulary_list=('Present', 'Absent'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [75]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(train_batch).numpy()[0])

[1. 0.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

Final Model

In [77]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adamax',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

print("--Fit model--")
model.fit(train_data, epochs=500)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(test_data, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%\n")

predictions = model.predict(test_data)

for prediction, hasCHD in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted CHD Diagnose Rate: {:.2%}".format(prediction[0]),
        "    | Actual outcome: ",
        ("Has CHD" if bool(hasCHD) else "no CHD"))

--Fit model--
Epoch 1/500
34/34 [==============================] - 1s 31ms/step - loss: 0.6603 - accuracy: 0.6131
Epoch 2/500
34/34 [==============================] - 0s 3ms/step - loss: 0.6050 - accuracy: 0.6548
Epoch 3/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5833 - accuracy: 0.6726
Epoch 4/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5777 - accuracy: 0.6964
Epoch 5/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5614 - accuracy: 0.6845
Epoch 6/500
34/34 [==============================] - 0s 4ms/step - loss: 0.5555 - accuracy: 0.7292
Epoch 7/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5474 - accuracy: 0.7321
Epoch 8/500
34/34 [==============================] - 0s 4ms/step - loss: 0.5395 - accuracy: 0.7083
Epoch 9/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5378 - accuracy: 0.7500
Epoch 10/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5305 - accuracy: 0